#Merging, Cleaning Data Sets and Adding Variables#

This notebook will be used to explore merging the data sets, cleaning the data sets (changing various values to np.NaN, converting fields to the appropriate value type), and adding calculated variables. The cleaned data st is exported as csv to be used in other data analysis.

In [338]:
import pandas as pd
import numpy as np

cross = pd.read_csv('000_Cross-Sectional.csv', low_memory=False)
base = pd.read_csv('00_Baseline.csv', low_memory=False)

pd.set_option('display.max_rows', 120)
cross.rename(columns={'ID':'SWANID'}, inplace=True)

##Merging Data Sets##

In [339]:
data = pd.merge(cross, base)

In [340]:
data.shape

(3302, 852)

That's a lot of fields.

##Cleaning Data Sets##

###Changing coded values for missing/null data to np.NaN###

In [341]:
data.replace(' ', np.nan, inplace=True)
data.replace('-9', np.nan, inplace=True)
data.replace('-1', np.nan, inplace=True)
data.replace('-7', np.nan, inplace=True)
data.replace('-8', np.nan, inplace=True)

###Correcting data types###

Discrimination scores

In [342]:
data[['COURTES0', 'RESPECT0', 'POORSER0', 'NOTSMAR0', 'AFRAIDO0', 'DISHONS0', 'BETTER0', 'INSULTE0', 
      'HARASSE0', 'IGNORED0']] = data[['COURTES0', 'RESPECT0', 'POORSER0', 'NOTSMAR0', 'AFRAIDO0', 'DISHONS0', 
                                       'BETTER0', 'INSULTE0', 'HARASSE0', 'IGNORED0']].astype(float)

Ages, BMI, P_STRESS, CRPRESU0

In [343]:
data[['AGE0']] = data[['AGE0']].astype(float)
data[['BMI0']] = data[['BMI0']].astype(float)
data[['P_STRESS']] = data[['P_STRESS']].astype(float)
data[['CRPRESU0']] = data[['CRPRESU0']].astype(float)
data[['HEART']] = data[['HEART']].astype(float)
data[['STROKE0']] = data[['STROKE0']].astype(float)
data[['INCOME0']] = data[['INCOME0']].astype(float)
data[['DEGREE']] = data[['DEGREE']].astype(float)
data[['ETHNIC']] = data[['ETHNIC']].astype(float)

###Adding Calculated Variables###

In [344]:
def convert_binary(cols):
    data[cols] = data[cols].map({'1':0, '2':1})
    # data[cols].replace(np.nan, 0, inplace=True)
    print data[cols].value_counts(dropna=False)

def convert_binary_float(cols):
    data[cols] = data[cols].map({1:0, 2:1})
    # data[cols].replace(np.nan, 0, inplace=True) # this may need to come out for certain variables
    print data[cols].value_counts(dropna=False)

In [345]:
data.STROKE0.replace(1, 0, inplace=True)
data.STROKE0.replace(2, 1, inplace=True)

data.HEART.replace(1, 0, inplace=True)
data.HEART.replace(2, 1, inplace=True)

*Average Discrimination Score*

In [346]:
data.loc[:, 'DISC_SCORE0'] = 5 - data[['COURTES0', 'RESPECT0', 'POORSER0', 'NOTSMAR0', 'AFRAIDO0', 'DISHONS0', 
                                   'BETTER0', 'INSULTE0', 'HARASSE0', 'IGNORED0']].mean(axis=1)

*Reason for Discrimination*

In [347]:
data['RACE_REASON0'] = data.MAINREA0.map({'1':1, '2':1, '3':0, '4':0, '5':0, '6':0, '7':0, '8':0, '9':0})

*Outcome measures*

In [348]:
# Difference between Lipid Vascular Age and actual age

data['LV_AGE_DIFF0'] = data.LV_AGE0 - data.AGE0

# Difference between BMI Vascular Age and actual age

data['BV_AGE_DIFF0'] = data.BV_AGE0 - data.AGE0


# % change between Lipid Vascular Age and actual age

data['LV_AGE_PCT0'] = (data.LV_AGE0 - data.AGE0) / data.AGE0

# % change between BMI Vascular Age and actual age

data['BV_AGE_PCT0'] = (data.BV_AGE0 - data.AGE0) / data.AGE0


###Dealing with Null Values###

1) Remove all records that are missing either a discrimination score or a lipid vascular age

In [349]:
data.shape

(3302, 858)

In [350]:
data.DISC_SCORE0.isnull().sum()

6

In [351]:
data.LV_AGE0.isnull().sum()

35

In [352]:
data.dropna(subset=['DISC_SCORE0', 'LV_AGE0'], inplace=True)

In [353]:
data.shape

(3262, 858)

2) Replacing null values with average for continuous variables (BMI, CRPRESU0, P_STRESS)

In [354]:
#data.dropna(subset=['BMI0', 'INCOME0', 'DEGREE', 'CRPRESU0', 'P_STRESS', 'HEART', 'STROKE0'], inplace=True)

In [355]:
# replace depending on race
# means = data.groupby('ETHNIC').P_STRESS.mean()
# for i in data[data.P_STRESS.isnull()].index:
#    df.loc[i, 'P_STRESS'] = means[df.loc[i].ETHNIC]

# replace with the average value for all null values

data.P_STRESS.fillna(data.P_STRESS.mean(), inplace=True)
data.BMI0.fillna(data.BMI0.mean(), inplace=True)
data.CRPRESU0.fillna(data.CRPRESU0.mean(), inplace=True)

3) Check null values for Income, Degree, Heart, and Stroke0

In [356]:
data.INCOME0.isnull().sum()

84

In [357]:
data.DEGREE.isnull().sum()

31

In [358]:
data.HEART.isnull().sum()

53

In [359]:
data.STROKE0.isnull().sum()

2

4) Replacing the null values for Income, Degree, Heart, and Stroke0, with the most frequent value

In [360]:
data.INCOME0.replace(np.nan, 3, inplace=True)

In [361]:
data.DEGREE.replace(np.nan, 3, inplace=True)

In [362]:
data.HEART.replace(np.nan, 0, inplace=True)

In [363]:
data.STROKE0.replace(np.nan, 0, inplace=True)

###Replacing category numbers with names###

Race

In [364]:
data['RACE'] = data.ETHNIC.map({1:'Black', 8:'Asian', 9:'Asian', 10:'Caucasian', 13:'Hispanic'})

In [365]:
data.RACE.value_counts(dropna=False)

Caucasian    1529
Black         922
Asian         529
Hispanic      282
Name: RACE, dtype: int64

BMI (putting into categories)

In [366]:
data['BMI_CAT'] = data.BMI0

data.loc[data.BMI_CAT < 18, ['BMI_CAT']] = '0_Underweight'
data.loc[data.BMI_CAT < 25, ['BMI_CAT']] = '1_HealthyWeight'
data.loc[data.BMI_CAT < 30, ['BMI_CAT']] = '2_Overweight'
data.loc[data.BMI_CAT < 100, ['BMI_CAT']] = '3_Obese'

###Writing to a new CSV###

In [367]:
data.to_csv('991_CleanedData.csv', index = False)

In [368]:
data.shape

(3262, 860)